In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from torch.utils.data import DataLoader , Dataset
from PIL import Image, ImageFile

## Создание данных

In [2]:
train_data_path = './train/'

In [3]:
# Проверка доступности CUDA
print(torch.cuda.is_available())  # Должно вывести True, если CUDA доступна

# Проверка устройства (если CUDA доступна)
if torch.cuda.is_available():
    print(f"CUDA is available. Device name: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

True
CUDA is available. Device name: NVIDIA GeForce GTX 1660 Ti


In [4]:
transform = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std = [0.229,0.224,0.225])
])
train_data = torchvision.datasets.ImageFolder(root = train_data_path, transform=transform)

In [5]:
val_data_path = './val/'
val_data = torchvision.datasets.ImageFolder(root = val_data_path, transform= transform)

In [6]:
test_data_path = './test/'
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)

In [7]:
batch_size = 64
train_data_loader = DataLoader(train_data, batch_size=batch_size,shuffle = True)
val_data_loader = DataLoader(val_data, batch_size= batch_size)
test_data_loader = DataLoader(test_data, batch_size= batch_size)

In [8]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size = 3, stride =1 ,padding = 1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16,32, kernel_size =3 ,stride=1, padding = 1)

        
        self.fc1 = nn.Linear(32 * 16 * 16, 50)
        self.fc2 = nn.Linear(50,2)
    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x),dim=1)
        return x

In [9]:
simple_net = SimpleNet()

dummy_input = torch.rand(1,3,64,64)

torch.onnx.export(
    simple_net, 
    dummy_input, 
    'simplenet.onnx', 
    verbose= True, 
    input_names=['input'],
    output_names=['output'])

In [10]:
optimizer = optim.Adam(simple_net.parameters(),lr=0.0001)

In [11]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
simple_net.to(device)

SimpleNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=8192, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=2, bias=True)
)

In [12]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=10,device='cpu'):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
            
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                inputs = inputs.to(device)
                targets = targets.to(device)
                
                output = model(inputs)
                loss = loss_fn(output,targets) 
                valid_loss += loss.data.item() * inputs.size(0)
                
                correct = torch.eq(torch.max(output, dim=1)[1], targets).view(-1)
                num_correct += torch.sum(correct).item()
                num_examples += correct.shape[0]
                
            valid_loss /= len(val_loader.dataset)

        print(f'Epoch: {epoch + 1}, Training Loss: {training_loss:.2f}, Validation Loss: {valid_loss:.2f}, Accuracy: {num_correct / num_examples:.2f}')

In [13]:
train(simple_net,optimizer,nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=10, device=device)

Epoch: 1, Training Loss: 0.68, Validation Loss: 0.77, Accuracy: 0.26
Epoch: 2, Training Loss: 0.64, Validation Loss: 0.63, Accuracy: 0.68
Epoch: 3, Training Loss: 0.62, Validation Loss: 0.66, Accuracy: 0.62
Epoch: 4, Training Loss: 0.60, Validation Loss: 0.69, Accuracy: 0.55
Epoch: 5, Training Loss: 0.58, Validation Loss: 0.54, Accuracy: 0.78
Epoch: 6, Training Loss: 0.57, Validation Loss: 0.70, Accuracy: 0.51
Epoch: 7, Training Loss: 0.55, Validation Loss: 0.61, Accuracy: 0.70
Epoch: 8, Training Loss: 0.54, Validation Loss: 0.58, Accuracy: 0.74
Epoch: 9, Training Loss: 0.53, Validation Loss: 0.62, Accuracy: 0.64
Epoch: 10, Training Loss: 0.52, Validation Loss: 0.64, Accuracy: 0.63


In [18]:
labels = ['cat','fish']

img = Image.open("test/cat/298845523_84a02f7bac.jpg") 

img = transform(img)  # (C, H, W)
img = img.unsqueeze(0).to(device) 

simple_net.eval()

with torch.no_grad():
    prediction = simple_net(img)  # (1, num_classes)
    prediction = F.softmax(prediction, dim=1)
    prediction = prediction.argmax(dim=1).item() 

print(labels[prediction])

cat
